In [ ]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
# os.environ['JAVA_HOME']='/usr'
# os.environ['SPARK_HOME'] ='/usr/local/lib/python3.9/site-packages/pyspark/'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
# os.environ['HADOOP_USER_NAME']='skhalikov'
os.environ['HADOOP_USER_NAME']='student02'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

In [ ]:
conf = SparkConf().setAppName('Explain_Plan_20240517')\
    .setMaster('yarn') \
    .set('spark.sql.adaptive.enabled', 'false')\
    .set('spark.sql.adaptive.coalescePartitions.parallelismFirst', 'true')\
    .set('spark.sql.shuffle.partitions', '220')

    # .set('spark.sql.adaptive.coalescePartitions.initialPartitionNum', '250')\
    # .set('spark.default.parallelism', '4')\
    # .set('spark.dynamicAllocation.enabled', 'false')\
    # .set("spark.sql.sources.partitionOverwriteMode", "dynamic") 
 # Required for overwriting ONLY the required partitioned folders, and not the entire root folder

spark = SparkSession.builder.config(conf=conf) \
    .getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(50), sc.applicationId)
# print("".ljust(50), spark.conf.get(""))

In [ ]:
import random
from datetime import date, timedelta

end = date.today()
start = end + timedelta(days=-300)

names = ['Bob', 'James', 'Marek', 'Johannes', 'Иванов', 'Смирнов', 'Кузнецов', 'Попов', 'Васильев', 'Петров', 'Соколов', 'Михайлов', 'Новиков', 'Федоров', 'Морозов', 'Волков', 'Алексеев', 'Лебедев', 'Семенов', 'Егоров', 'Павлов', 'Козлов', 'Степанов', 'Николаев']
countries = ['rus:eu', 'bel:eu', 'usa:am', 'fra:eu', 'aus:as', 'ger:eu', 'swe:eu', 'fin:eu', 'chi:as']
customers = list()

# save_path = "/user/student02/customers_partitioned_year_month"
sc.setJobDescription('MY DESCRIPTION JOB 1')

In [ ]:
length_customers = 1000
for i in range(length_customers):
    random_date = start + (end - start) * random.random()

    if i % 3 == 0:
        country = countries[-1] # chi - 'China'
    else:
        country = countries[random.randint(0, len(countries)-2)] # without China
    
    customer = (names[random.randint(0, len(names)-1)], country, random.randint(-1000000, 1000000), random_date)
    customers.append(customer)

df_customers = spark.createDataFrame(data=customers, schema=["name","country","balance","last_order_date"])
    # .withColumn("year", F.year("last_order_date"))\
    # .withColumn("month", F.month("last_order_date"))
# df_customers.explain(True)

In [ ]:
# print(df_customers.rdd.getNumPartitions())
df_customers.createOrReplaceTempView("customers")

In [ ]:
# df_to_explain = df_customers.groupBy("country").agg(F.count("name"))
# sc.setJobDescription('MY JOB group by rollup(country)')
# sql_clause = " select country, count(1), sum(balance) from customers \
#                              group by rollup(country) order by country desc"
# df_to_explain = spark.sql(sql_clause).show()
# spark.sql("EXPLAIN " + sql_clause).show(60, False, False)
# .orderBy("name", "year", "month") #.show()
# df_grouped.rdd.getNumPartitions()
# df_to_explain.explain(True)

In [ ]:
sc.setJobDescription('JOB avg(balance) by name ')
df_to_explain3 = spark.sql("select name, avg(balance) avgbal from customers group by name")
df_to_explain3.collect()

In [ ]:
# df_to_explain.collect()
print(df_to_explain3.rdd.getNumPartitions())


In [ ]:
--

In [ ]:
spark.stop()